# 🐱🐶 Cat/Dog Classifier with MLflow Tracking

This notebook demonstrates MLOps best practices:
- **ResNet-18** fine-tuning for cat/dog classification
- **MLflow** for experiment tracking (params, metrics, artifacts)
- Comprehensive metrics: Accuracy, Precision, Recall, F1, AUC-ROC

## How to Use
1. Run all cells to train and log to MLflow
2. Open MLflow UI: `mlflow ui` in terminal
3. Compare experiments at http://localhost:5000

In [ ]:
# ==========================================
# CONFIGURATION - Modify these for experiments
# ==========================================

# Hyperparameters
LEARNING_RATE = 0.001      # Try: 0.001, 0.0001, 0.00001
EPOCHS = 5                 # Try: 5, 10, 15
BATCH_SIZE = 16            # Try: 8, 16, 32

# Model configuration
UNFREEZE_LAYER4 = False    # Set True for Phase 3 (more data)

# Data version tag (for tracking)
DATA_VERSION = "v1"        # Update when data changes: v1, v2, etc.

In [ ]:
# ==========================================
# IMPORTS
# ==========================================
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import mlflow
import mlflow.pytorch
from sklearn.metrics import (
    confusion_matrix, classification_report,
    precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve
)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

print(f"PyTorch version: {torch.__version__}")
print(f"MLflow version: {mlflow.__version__}")
print(f"Device: {'cuda' if torch.cuda.is_available() else 'cpu'}")

In [ ]:
# ==========================================
# DATA LOADING
# ==========================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Transforms for ResNet-18
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load dataset from data/ folder
dataset = datasets.ImageFolder('data/', transform=transform)
print(f"Classes: {dataset.classes}")
print(f"Total images: {len(dataset)}")

# Train/Val split (80/20)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(
    dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
# ==========================================
# MODEL SETUP
# ==========================================

# Load pretrained ResNet-18
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Optionally unfreeze layer4 (for Phase 3 with more data)
if UNFREEZE_LAYER4:
    for param in model.layer4.parameters():
        param.requires_grad = True
    print("✓ Layer4 UNFROZEN - fine-tuning enabled")
else:
    print("✓ All layers FROZEN - feature extraction only")

# Replace classifier for binary classification (Cat=0, Dog=1)
model.fc = nn.Linear(model.fc.in_features, 2)

# Move to device
model = model.to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# ==========================================
# TRAINING FUNCTIONS
# ==========================================

def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / total, correct / total


def evaluate(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            probs = torch.softmax(outputs, dim=1)
            _, predicted = outputs.max(1)
            
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())  # Prob of Dog class
    
    return {
        'loss': running_loss / len(loader.dataset),
        'labels': np.array(all_labels),
        'predictions': np.array(all_preds),
        'probabilities': np.array(all_probs)
    }

In [ ]:
# ==========================================
# VISUALIZATION FUNCTIONS
# ==========================================

def plot_confusion_matrix(labels, predictions, classes):
    """Create and save confusion matrix plot."""
    cm = confusion_matrix(labels, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=classes, yticklabels=classes)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png', dpi=150)
    plt.close()
    return 'confusion_matrix.png'


def plot_roc_curve(labels, probabilities):
    """Create and save ROC curve plot."""
    fpr, tpr, _ = roc_curve(labels, probabilities)
    auc = roc_auc_score(labels, probabilities)
    
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC (AUC = {auc:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
    plt.xlim([0, 1])
    plt.ylim([0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.savefig('roc_curve.png', dpi=150)
    plt.close()
    return 'roc_curve.png'


def plot_training_curves(train_losses, val_losses, val_accs):
    """Create and save training curves plot."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    epochs = range(1, len(train_losses) + 1)
    
    ax1.plot(epochs, train_losses, 'b-', label='Train Loss')
    ax1.plot(epochs, val_losses, 'r-', label='Val Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Loss Curves')
    ax1.legend()
    
    ax2.plot(epochs, val_accs, 'g-', label='Val Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.set_title('Validation Accuracy')
    ax2.legend()
    
    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=150)
    plt.close()
    return 'training_curves.png'

In [ ]:
# ==========================================
# TRAINING WITH MLFLOW TRACKING
# ==========================================

# Set experiment name
mlflow.set_experiment("Cat-Dog-Classifier")

# Create run name with timestamp
run_name = f"resnet18-{DATA_VERSION}-{datetime.now().strftime('%H%M')}"

with mlflow.start_run(run_name=run_name):
    # ===== LOG PARAMETERS =====
    mlflow.log_params({
        "learning_rate": LEARNING_RATE,
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "model": "resnet18",
        "pretrained": True,
        "unfreeze_layer4": UNFREEZE_LAYER4,
        "data_version": DATA_VERSION,
        "train_samples": len(train_dataset),
        "val_samples": len(val_dataset),
        "total_params": total_params,
        "trainable_params": trainable_params
    })
    
    # Tags for organization
    mlflow.set_tag("task", "binary_classification")
    mlflow.set_tag("framework", "pytorch")
    
    # Setup training
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=LEARNING_RATE
    )
    
    # Training history
    train_losses, val_losses, val_accs = [], [], []
    best_val_acc = 0.0
    
    print(f"\n{'='*50}")
    print(f"Training: {run_name}")
    print(f"Data: {len(train_dataset)} train, {len(val_dataset)} val")
    print(f"{'='*50}\n")
    
    # ===== TRAINING LOOP =====
    for epoch in range(EPOCHS):
        # Train
        train_loss, train_acc = train_one_epoch(
            model, train_loader, criterion, optimizer
        )
        
        # Evaluate
        val_results = evaluate(model, val_loader, criterion)
        val_loss = val_results['loss']
        val_acc = (val_results['predictions'] == val_results['labels']).mean()
        
        # Store history
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_accs.append(val_acc)
        
        # Log metrics per epoch
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc
        }, step=epoch)
        
        # Track best
        if val_acc > best_val_acc:
            best_val_acc = val_acc
        
        print(f"Epoch {epoch+1}/{EPOCHS} | "
              f"Train Loss: {train_loss:.4f} | "
              f"Val Loss: {val_loss:.4f} | "
              f"Val Acc: {val_acc:.4f}")
    
    # ===== FINAL EVALUATION =====
    final_results = evaluate(model, val_loader, criterion)
    labels = final_results['labels']
    preds = final_results['predictions']
    probs = final_results['probabilities']
    
    # Calculate comprehensive metrics
    accuracy = (preds == labels).mean()
    precision = precision_score(labels, preds, average='binary')
    recall = recall_score(labels, preds, average='binary')
    f1 = f1_score(labels, preds, average='binary')
    auc = roc_auc_score(labels, probs)
    
    # Log final metrics
    mlflow.log_metrics({
        "final_accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "auc_roc": auc,
        "best_val_acc": best_val_acc
    })
    
    print(f"\n{'='*50}")
    print("FINAL METRICS")
    print(f"{'='*50}")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"AUC-ROC:   {auc:.4f}")
    
    # ===== LOG ARTIFACTS =====
    # Confusion Matrix
    cm_path = plot_confusion_matrix(labels, preds, dataset.classes)
    mlflow.log_artifact(cm_path)
    print(f"\n✓ Logged: {cm_path}")
    
    # ROC Curve
    roc_path = plot_roc_curve(labels, probs)
    mlflow.log_artifact(roc_path)
    print(f"✓ Logged: {roc_path}")
    
    # Training Curves
    curves_path = plot_training_curves(train_losses, val_losses, val_accs)
    mlflow.log_artifact(curves_path)
    print(f"✓ Logged: {curves_path}")
    
    # Save and log model
    os.makedirs('models', exist_ok=True)
    model_path = f'models/model_{DATA_VERSION}.pth'
    torch.save(model.state_dict(), model_path)
    mlflow.log_artifact(model_path)
    print(f"✓ Logged: {model_path}")
    
    # Log model with MLflow format
    mlflow.pytorch.log_model(model, "pytorch_model")
    print(f"✓ Logged: MLflow PyTorch model")
    
    print(f"\n{'='*50}")
    print(f"Run ID: {mlflow.active_run().info.run_id}")
    print(f"View at: mlflow ui")
    print(f"{'='*50}")

In [ ]:
# ==========================================
# CLASSIFICATION REPORT
# ==========================================
print("\nDetailed Classification Report:")
print(classification_report(labels, preds, target_names=dataset.classes))

---

## 📋 Next Steps

### Phase 2: Hyperparameter Tuning
1. Change `LEARNING_RATE` to `0.0001`
2. Change `EPOCHS` to `10`
3. Run all cells again
4. Compare runs in MLflow UI

### Phase 3: More Data + Unfreeze Layer4
1. Add more images to `data/Cat/` and `data/Dog/`
2. Run: `dvc add data/` and `git commit`
3. Set `UNFREEZE_LAYER4 = True`
4. Update `DATA_VERSION = "v2"`
5. Run all cells again
6. Compare all runs in MLflow UI